In [1]:
# Import OS to interact with local computer operating system
import sys
import requests
from types import ModuleType
# Import the datetime and date classes from the datetime module
from datetime import datetime, date

# Import the Pandas library for data manipulation and analysis, aliased as pd.
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

# Import the NumPy library for data manipulation and analysis, aliased as np.
try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

In [2]:
# Define the URL of the Python module to be downloaded from Dropbox.
# The 'dl=1' parameter in the URL forces a direct download of the file content.
url= 'https://www.dropbox.com/scl/fi/4y5hjxlfphh1ngvbgo77q/\
module_-basic_concepts_fixed_income.py?rlkey=6oxi7mgka42veaat79hcv8boz&st=87sztshr&dl=1'
module_name='basic_concepts_fixed_income'
# Send an HTTP GET request to the URL and store the server's response.
try:
  response=requests.get(url)
  # Raise an exception for bad status codes (like 404 Not Found)
  response.raise_for_status()
  module= ModuleType(module_name)
  #Code contained in response.text executed
  exec(response.text, module.__dict__)
  # Module added to sys
  sys.modules[module_name]=module
except requests.exceptions.RequestException as e:
    print(f"❌ Error: Could not fetch module from URL. {e}")
except Exception as e:
    print(f"❌ Error: Failed to execute or import the module. {e}")

# Now that 'basic_concepts_fixed_income' exists in the notebook, import the specific functions
from basic_concepts_fixed_income import (accrued_interest,
                                         create_payoff_matrix,
                                         bond_pay_data)

In [3]:
#The full file path.
url='https://www.dropbox.com/scl/fi/lgnaj41bt8o9sv5a63rr1/\
bond_data_jan21_2025.xlsx?rlkey=twjzkcqo0g2ahvot78518ti4x&st=ihc5feog&dl=1'
print(f"Attempting to load data from: {url}")

#Load the data from Excel, using the first column as the index.
try:
    bond_data = pd.read_excel(url, index_col='Maturity Date',sheet_name='Fidelity Data')

    # Display the first and last 5 rows of the loaded DataFrame to verify it worked.
    display(bond_data)

except FileNotFoundError:
    print("\nERROR: File not found.")
    print("Please check that the 'folder' and 'file' variables are spelled correctly'\
' and that the file exists in that location.")

Attempting to load data from: https://www.dropbox.com/scl/fi/lgnaj41bt8o9sv5a63rr1/bond_data_jan21_2025.xlsx?rlkey=twjzkcqo0g2ahvot78518ti4x&st=ihc5feog&dl=1


,Description,Coupon,Price Bid,Price Ask,Bid Size,Ask Size
Maturity Date,,,,,,
2025-01-28,UNITED STATES TREAS BILLS ZERO CPN 0.000...,0.000,99.929,99.930,100000.0,100000.0
2025-01-30,UNITED STATES TREAS BILLS ZERO CPN 0.000...,0.000,99.906,99.907,40000.0,40000.0
2025-01-31,UNITED STATES TREAS SER U-2025 1.3750...,1.375,99.921,99.934,60000.0,60000.0
2025-01-31,UNITED STATES TREAS SER AW-2025 4.1250...,4.125,99.988,99.997,60000.0,60000.0
2025-01-31,UNITED STATES TREAS SER G-2025 2.5000...,2.500,99.953,99.965,60000.0,60000.0
...,...,...,...,...,...,...
2029-11-30,UNITED STATES TREAS SER AG-2029 4.1250...,4.125,98.910,98.914,100000.0,100000.0
2029-11-30,UNITED STATES TREAS SER S-2029 3.8750...,3.875,97.763,97.782,40000.0,40000.0
2029-12-31,UNITED STATES TREAS SER T-2029 3.8750...,3.875,97.734,97.738,65000.0,65000.0


In [4]:
settlement=date(2025,1,21)
# apply the accrued_interest function to the DataFrame
bond_data['Accrued'] = bond_data.apply(
    lambda x: accrued_interest(
        maturity=x.name, # Assuming index is maturity
        coupon=x['Coupon'],
        settlement=settlement,
        freq=2,
        day_type='Actual/Actual'
    ), axis=1
)

# Calculate transaction prices
transaction_prices=((bond_data['Price Bid']+bond_data['Price Ask'])/2+
                    bond_data['Accrued']).to_numpy()

# Create payoff matrix
payoffs_304,column_dates_304=create_payoff_matrix(bond_data,settlement=settlement,freq=2)

# Estimate the present value factors with least squares
zero_prices_304,ss2_304,rank_304,colInfo_304=np.linalg.lstsq(payoffs_304,transaction_prices)

# Maturity of the bonds in years
maturity_years=[(mat_date-settlement).days/365.25 for mat_date in column_dates_304]


# Continuously compounded annualized spot rates
spot_rates_304=-np.log(zero_prices_304)/np.array(maturity_years)

# Continuously compounded annualized forward rates
forward_rates_304=-np.diff(np.log(zero_prices_304))/np.diff(maturity_years)
forward_rates_304=np.insert(forward_rates_304,0,np.nan)

# Create a DataFrame of least squares present value estimates and display results
# First a dictionary
bonds_304={'Zero Prices: 304':zero_prices_304,
           'Spot Rates:304':spot_rates_304,
           'Forward Rates: 304':forward_rates_304}

# Then a DataFrame
df_304=pd.DataFrame(bonds_304,index=column_dates_304)
display(df_304)

# Create a DataFrame of summary statistics and display results
# First a dictionary
summary_304={'Average Sum Squared Errors':ss2_304/len(bond_data),
             'Payoff Rank':rank_304,
             'Column Max Info':np.max(colInfo_304),
             'Column Min Info':np.min(colInfo_304)}
# Then a DataFrame
df_summary_304=pd.DataFrame(summary_304,index=['Summary Statistics']).T
display(df_summary_304)

/tmp/ipykernel_13487/3492082812.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  zero_prices_304,ss2_304,rank_304,colInfo_304=np.linalg.lstsq(payoffs_304,transaction_prices)


,Zero Prices: 304,Spot Rates:304,Forward Rates: 304
2025-01-28,0.999295,0.036799,NaN
2025-01-30,0.999065,0.037963,0.042038
2025-01-31,0.998886,0.040711,0.065439
2025-02-04,0.998475,0.039817,0.037581
2025-02-06,0.998240,0.040213,0.042987
...,...,...,...
2029-10-31,0.812875,0.043390,0.043360
2029-11-15,0.809539,0.043874,0.100158
2029-11-30,0.809936,0.043402,-0.011943
2029-12-31,0.806965,0.043400,0.043302


,Summary Statistics
Average Sum Squared Errors,0.011753
Payoff Rank,149.000000
Column Max Info,267.431046
Column Min Info,100.000000


In [5]:
# list assumed settlement dates
coupon=0.01
settlement=date(2025,1,21)
assumed_settlement_dates=[bond_pay_data(date(2025,6,30),
                          coupon,settlement=settlement)[0][0],
                          bond_pay_data(date(2025,4,30),
                          coupon,settlement=settlement)[0][0]]

assumed_settlement_dates

[datetime.date(2025, 6, 30), datetime.date(2025, 4, 30)]

In [6]:
# pay_date list
forward_zeros=[]

# maturity are from the bond, any number greater than zero works for coupon
coupon=0.01

# list maturitty dates
maturity_dates=[date(2029,12,31),date(2029,10,31)]

for maturity,assumed_settlement_date in zip(maturity_dates,assumed_settlement_dates):
  pay_dates,_=bond_pay_data(maturity,coupon,settlement=assumed_settlement_date,freq=2)
  matching_zero_prices=df_304['Zero Prices: 304'].loc[pay_dates]
  normalizing_zero=df_304['Zero Prices: 304'].loc[assumed_settlement_date]
  forward_zeros.append((matching_zero_prices)/normalizing_zero)

display(forward_zeros)

[2025-12-31    0.979271
 2026-06-30    0.958857
 2026-12-31    0.938939
 2027-06-30    0.918966
 2028-01-03    0.898935
 2028-06-30    0.879044
 2029-01-02    0.859154
 2029-07-02    0.840161
 2029-12-31    0.822042
 Name: Zero Prices: 304, dtype: float64,
 2025-10-31    0.979291
 2026-04-30    0.958846
 2026-11-02    0.938948
 2027-04-30    0.919132
 2027-11-01    0.899442
 2028-05-01    0.879359
 2028-10-31    0.859813
 2029-04-30    0.840726
 2029-10-31    0.822358
 Name: Zero Prices: 304, dtype: float64]

In [7]:
coupon=0.01

# validate the assumed settlement date
assumed_settlement,_=bond_pay_data(date(2025,1,31),coupon,settlement=settlement)

# maturity are from the bond, any number greater than zero works for coupon
# assumed settlement is a NumPy array with a single elemement of a valid assumed date
maturity=date(2030,1,31)
pay_dates,_=bond_pay_data(maturity,coupon,settlement=assumed_settlement[0],freq=2)

# loc attribute selects the matching zero price estimates
matching_zero_prices=pd.Series(df_304['Zero Prices: 304'].loc[pay_dates])

# loc attribute selects the zero price of assumed settlement date for normalization
normalizing_zero_price= df_304['Zero Prices: 304'].loc[assumed_settlement[0]]

# forward zero prices calculated
forward_zeros.append(pd.Series(matching_zero_prices/normalizing_zero_price))

# forward zero prices for the three dates
display(forward_zeros)

[2025-12-31    0.979271
 2026-06-30    0.958857
 2026-12-31    0.938939
 2027-06-30    0.918966
 2028-01-03    0.898935
 2028-06-30    0.879044
 2029-01-02    0.859154
 2029-07-02    0.840161
 2029-12-31    0.822042
 Name: Zero Prices: 304, dtype: float64,
 2025-10-31    0.979291
 2026-04-30    0.958846
 2026-11-02    0.938948
 2027-04-30    0.919132
 2027-11-01    0.899442
 2028-05-01    0.879359
 2028-10-31    0.859813
 2029-04-30    0.840726
 2029-10-31    0.822358
 Name: Zero Prices: 304, dtype: float64,
 2025-07-31    0.979138
 2026-02-02    0.958690
 2026-07-31    0.938935
 2027-02-01    0.919015
 2027-08-02    0.899549
 2028-01-31    0.879982
 2028-07-31    0.860395
 2029-01-31    0.841154
 2029-07-31    0.822730
 2030-01-31    0.804344
 Name: Zero Prices: 304, dtype: float64]

In [8]:
# par yeilds for each of the three dates
dec_29=(1-forward_zeros[0])/forward_zeros[0].cumsum()*2
oct_29=(1-forward_zeros[1])/forward_zeros[1].cumsum()*2
jan_30=(1-forward_zeros[2])/forward_zeros[2].cumsum()*2

# combine and sort
par_yields=pd.concat([dec_29,oct_29,jan_30]).sort_index()

# display sorted results
display(par_yields)

2025-07-31    0.042613
2025-10-31    0.042295
2025-12-31    0.042335
2026-02-02    0.042635
2026-04-30    0.042467
2026-06-30    0.042456
2026-07-31    0.042454
2026-11-02    0.042440
2026-12-31    0.042447
2027-02-01    0.042671
2027-04-30    0.042605
2027-06-30    0.042694
2027-08-02    0.042788
2027-11-01    0.042830
2028-01-03    0.043052
2028-01-31    0.043053
2028-05-01    0.043279
2028-06-30    0.043400
2028-07-31    0.043385
2028-10-31    0.043571
2029-01-02    0.043788
2029-01-31    0.043658
2029-04-30    0.043783
2029-07-02    0.043952
2029-07-31    0.043773
2029-10-31    0.043874
2029-12-31    0.043965
2030-01-31    0.043948
Name: Zero Prices: 304, dtype: float64